In [9]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z**(k//(q+1))

In [10]:
def base_change_hermitian(U):
    Up = U.LU()[2]
    D = Up.diagonal()
    A = ~Up * matrix.diagonal([d.sqrt() for d in D])
    diag = (A.H * U * A).diagonal()
    factor_diag = diagonal_matrix([conj_square_root(d) for d in diag])
    return (factor_diag*A.inverse()).H

In [11]:
#use libgap.eval for GAP evalutation of BaseChangeToCanonical using `forms` package
from sage.libs.gap.libgap import libgap
def base_change_hermitian_gap_forms(U,q):
    if U.nrows() == 1 and U.ncols() == 1:
        return matrix(F,[[factor_scalar(U[0,0])]])
    loaded_forms = libgap.LoadPackage("forms")
    return matrix(F,libgap.BaseChangeToCanonical(libgap([list(row) for row in U]).HermitianFormByMatrix(F))).inverse()

In [12]:
def base_change_hermitian_gap_source(mat):
    """
    Diagonalizes a Hermitian matrix over a finite field.
    Returns the base change matrix and the rank of the Hermitian form.
    
    Arguments:
        mat: The Gram matrix of a Hermitian form (Sage matrix object)
        F: The finite field (GF(q))
    
    Returns:
        D: The base change matrix
        r: The number of non-zero rows in D*mat*D^T
    """

    F = mat.base_ring()
    n = mat.nrows()
    q = sqrt(F.order())

    if mat.nrows() == 1 and mat.ncols() == 1:
        return matrix(F,[conj_square_root(mat[0][0])])

    A = copy(mat)
    D = identity_matrix(F, n)
    row = 0

    # Diagonalize A
    while True:
        row += 1

        # Look for a non-zero element on the main diagonal, starting from `row`
        i = row - 1  # Adjust for zero-based indexing in Sage
        while i < n and A[i, i].is_zero():
            i += 1

        if i == row - 1:
            # Do nothing since A[row, row] != 0
            pass
        elif i < n:
            # Swap to ensure A[row, row] != 0
            A.swap_rows(row - 1, i)
            A.swap_columns(row - 1, i)
            D.swap_rows(row - 1, i)
        else:
            # All entries on the main diagonal are zero; look for an off-diagonal element
            i = row - 1
            while i < n - 1:
                k = i + 1
                while k < n and A[i, k].is_zero():
                    k += 1
                if k == n:
                    i += 1
                else:
                    break

            if i == n - 1:
                # All elements are zero; terminate
                row -= 1
                r = row
                break

            # Fetch the non-zero element and place it at A[row, row + 1]
            if i != row - 1:
                A.swap_rows(row - 1, i)
                A.swap_columns(row - 1, i)
                D.swap_rows(row - 1, i)

            A.swap_rows(row, k)
            A.swap_columns(row, k)
            D.swap_rows(row, k)

            b = A[row, row - 1]**(-1)
            A.add_multiple_of_column(row - 1, row, b**q)
            A.add_multiple_of_row(row - 1, row, b)
            D.add_multiple_of_row(row - 1, row, b)

        # Eliminate below-diagonal entries in the current column
        a = -A[row - 1, row - 1]**(-1)
        for i in range(row, n):
            b = A[i, row - 1] * a
            if not b.is_zero():
                A.add_multiple_of_column(i,row - 1, b**q)
                A.add_multiple_of_row(i, row - 1, b)
                D.add_multiple_of_row(i, row - 1, b)

        if row == n - 1:
            break

    # Count how many variables have been used
    if row == n - 1:
        if not A[n - 1, n - 1].is_zero():
            r = n
        else:
            r = n - 1

    # Normalize diagonal elements to 1
    for i in range(r):
        a = A[i, i]
        if not a.is_one():
            # Find an element `b` such that `b*b^t = b^(t+1) = a`
            b = conj_square_root(a)
            D.rescale_row(i, 1 / b)

    return D.inverse()

In [13]:
q = 11
F = GF(q**2)
U=matrix(F,[[1,4,7],[4,1,4],[7,4,1]])
#U=matrix(F,[[1, 5, 2, 5, 2, 0],[5, 1, 5, 5, 0, 2],[2, 5, 1, 0, 5, 2],[5, 5, 0, 1, 5, 5],[2, 0, 5, 5, 1, 5],[0, 2, 2, 5, 5, 1]])

In [14]:
#use the upper part of the LU decomposition, and factor the diagonal
B = base_change_hermitian(U)
print(B*B.H == U)
print(B)

True
[   1    0    0]
[   4 8*z2    0]
[   7 4*z2 2*z2]


In [15]:
#compute the matrix factorization using libgap and the GAP `forms` package
B = base_change_hermitian_gap_forms(U,q)
print(B*B.H == U)
print(B)

True
[        1         0         0]
[        4  9*z2 + 2         0]
[        7 10*z2 + 1  3*z2 + 3]


In [16]:
#compute the matrix factorization using a direct translation of the GAP code into Sage
B = base_change_hermitian_gap_source(U)
print(B*B.H == U)
print(B)

True
[        1         0         0]
[        4  9*z2 + 2         0]
[        7 10*z2 + 1  3*z2 + 3]


In [17]:
#counterexample for d_rho=3, q=3 by brute force
q = 3
F = GF(3**2)
U = matrix(F,[[0, 1, 2], [1, 0, 1], [2, 1, 0]])

In [18]:
#minimal code throwing error for 1x1 case for BaseChangeToCanonical
from sage.libs.gap.element import GAPError
try:
    libgap.LoadPackage("forms")
    libgap.BaseChangeToCanonical(libgap(matrix(GF(q**2),[[1]])).HermitianFormByMatrix(GF(q**2)))
except GAPError as e:
    print(e)

Error, row index 2 exceeds 1, the number of rows


In [19]:
# Generate all possible 3x3 upper triangular matrices (diagonal included) over F_9
# check if A*A.H == U
import itertools
def counterexample_check():
    A = matrix(F,3)
    num_iters = 0
    for combination in itertools.product(F, repeat=6):
        # Unpack the combination into the matrix entries
        a, b, c, d, e, f = combination
        
        # Construct the upper triangular matrix
        A = matrix(F,[
            [a, b, c],
            [0, d, e],
            [0, 0, f]
        ])
        if num_iters % 10_000 == 0:
            print(f"{round(100*num_iters/9**6,2)}%")
        
        if A*A.H == U:
            print("found a decomposition U = AA^*: ",A)
        num_iters += 1

In [20]:
#note we get a clear failure to factorize here and it's because a factorization does not exist
A_1 = base_change_hermitian(U)
print(A_1*A_1.H == U)
print(A_1)

False
[ 0  0  0]
[ 0  0  0]
[ 0  0 z2]


In [21]:
A_2 = base_change_hermitian_gap_source(U)
print(A_2*A_2.H == U)
print(A_2)

True
[2*z2    2    0]
[2*z2    1    0]
[   0    1   z2]


In [22]:
A_3 = base_change_hermitian_gap_forms(U,q)
print(A_3*A_3.H == U)
print(A_3)

True
[2*z2 + 1 2*z2 + 2        0]
[       1       z2        0]
[    2*z2   z2 + 2       z2]
